# Combine dat files from unattended ApRES surveys on Thwaites glacier into netcdfs for archiving
### J. Kingslake, July 2nd 2025
The code below collates the dat files collected in unattneded ApRES surveys on Thwaites Glacier between 2022 and 2024 by the GHOST team, part of the International Thwaites Glacier Collaboration. The data were collected by Elizabeth Case, Columbia University, then Utrecht University, Andrew Hoffman, Columbia University, then Rice University, and Ole Zeisling, Alfred Wegner Institute.

In [1]:
import xapres as xa
import xarray as xr
import os

Define where the dat files are and where we will write the netcdfs.

In [2]:
sites = ["ApRES_LTG", "ApRES_Lake1", "ApRES_Lake2", "ApRES_Takahe1_204", "ApRES_Takahe2_203"]
source = "/Users/jkingslake/Documents/data/thwaites_apres/original/continuous/"
archive_location ='/Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended'

Define a function for changing the format of the attributes in the xarray to work nicely when written to netcdf. 

In [3]:
def remove_constants_attr(ds):
    for c in ds.attrs['constants']:
        ds.attrs[c] = ds.attrs['constants'][c]
    del ds.attrs['constants']
    return ds


Loop through the loacations and:
1. find all the dat files, subset them, and sort them by filename, so that they are in time order. 
2. Split the list of dat files into blocks with a size that results in reasonably sized netcdfs. 
3. Define netcdf paths and make directoies for them
4. Delete previously created netcdfs 
5. loop through the blocks of files and load all the dat files into an xarray using the package xapres
6. add the stacked chirps
7. tidy up the units and attributes to work well with netcdfs
8. write the netcdfs
(the last three steps are performed for each block of files) 

In [5]:

tg_1 = xa.load.from_dats()
for site in sites:
    # 1. find all the dat files, subset them, and sort them by filename, so that they are in time order. 
    dat_directory = os.path.join(source, site, "dat_files")
    filepaths = tg_1.list_files(directory=dat_directory)
    print(f"there are {len(filepaths)} files from {site}")
    one_dat = tg_1.load_all(directory = dat_directory, file_numbers_to_process=[7], computeProfiles = False, disable_progress_bar=True)
    print(f"total size of dataset without profiles computed = {one_dat.nbytes/1e9 * len(filepaths):.2f} GB in memory")

    if site == 'ApRES_LTG': 
        start_dat_file_number = 4  # the first 4 use the same attenuator settings, but they are not regularly spaces in time with the other data
    elif site == 'ApRES_Lake1':
        start_dat_file_number = 3  # the first 3 use different attenuator settings. THis is the first one which is regularly spaced in time with the other data
    elif site == 'ApRES_Lake2':
        start_dat_file_number = 4  # This is the first one which is regularly spaced in time with the other data
    elif site == 'ApRES_Takahe1_204':
        start_dat_file_number = 0
    elif site == 'ApRES_Takahe2_203':
        start_dat_file_number = 3 # something is up with the first 3 files some seem to have no data and one is the wrong size. 
    else:
        raise ValueError(f"Unknown site {site}")

    dat_size = os.path.getsize(filepaths[start_dat_file_number+5])
    print(f"dat file size = {dat_size/1e6:.2f} MB on disk")

    f = sorted(filepaths)[start_dat_file_number:]

    # 2. Split the list of dat files into blocks with a size that results in reasonably sized netcdfs. 
    N = int(17*104e6/dat_size)  # number of files to process in each block
    file_lists = [f[i:i + N] for i in range(0, len(f), N)]
    print(f'created {len(file_lists)} file lists from {len(f)} files')
    print(f'list sizes: {[len(x) for x in file_lists]}')

    # 3. Define netcdf paths and make directoies for them
    nc_locations = [f'{archive_location}/{site}/netcdf/part_{i}.nc' for i in range(len(file_lists))]
    os.makedirs(f"{archive_location}/{site}/netcdf", exist_ok=True) 
    
    # 4. Delete previously created netcdfs 
    for nc in nc_locations:
        if os.path.exists(nc):
            print(f"deleting existing file {nc}")
            os.remove(nc)

    # 5. loop through the blocks of files and load all the dat files into an xarray using the package xapres
    for i, file_list in enumerate(file_lists):
        print(f"Processing {len(file_list)} files")
        ds = tg_1.load_all(directory = dat_directory, 
                        computeProfiles = False,
                        file_names_to_process=file_list,
                        disable_progress_bar=False)
        
        # 6. add the stacked chirps
        ds['chirp_stacked'] = ds.chirp.mean(dim='chirp_num', keep_attrs=True)
        
        # 7. tidy up the units and attributes to work well with netcdfs
        ds.chirp_stacked.attrs['long_name'] = 'stacked de-ramped chirp'
        ds["chirp_time"].attrs["units"] = "unscaled seconds"
        ds = remove_constants_attr(ds)
        
        # 8. write the netcdfs
        print(f"Writing netcdf to {nc_locations[i]}")
        ds.to_netcdf(nc_locations[i])
    print(f"Finished processing {site} with {len(file_lists)} netcdfs created")
    print('------------------------------------------------------')


there are 181 files from ApRES_LTG
total size of dataset without profiles computed = 11.64 GB in memory
dat file size = 8.00 MB on disk
created 1 file lists from 177 files
list sizes: [177]
deleting existing file /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_LTG/netcdf/part_0.nc
Processing 177 files


100%|██████████| 177/177 [00:05<00:00, 32.33it/s]


Writing netcdf to /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_LTG/netcdf/part_0.nc
Finished processing ApRES_LTG with 1 netcdfs created
------------------------------------------------------
there are 185 files from ApRES_Lake1
total size of dataset without profiles computed = 11.90 GB in memory
dat file size = 16.00 MB on disk
created 2 file lists from 182 files
list sizes: [110, 72]
deleting existing file /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Lake1/netcdf/part_0.nc
deleting existing file /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Lake1/netcdf/part_1.nc
Processing 110 files


100%|██████████| 110/110 [00:04<00:00, 23.79it/s]


Writing netcdf to /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Lake1/netcdf/part_0.nc
Processing 72 files


...

Writing netcdf to /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Takahe1_204/netcdf/part_1.nc
Finished processing ApRES_Takahe1_204 with 2 netcdfs created
------------------------------------------------------
there are 34 files from ApRES_Takahe2_203
total size of dataset without profiles computed = 14.16 GB in memory
dat file size = 104.02 MB on disk
created 2 file lists from 31 files
list sizes: [16, 15]
deleting existing file /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Takahe2_203/netcdf/part_0.nc
deleting existing file /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Takahe2_203/netcdf/part_1.nc
Processing 16 files


100%|██████████| 16/16 [00:03<00:00,  4.97it/s]


Writing netcdf to /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Takahe2_203/netcdf/part_0.nc
Processing 15 files


100%|██████████| 15/15 [00:02<00:00,  5.31it/s]


Writing netcdf to /Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended/ApRES_Takahe2_203/netcdf/part_1.nc
Finished processing ApRES_Takahe2_203 with 2 netcdfs created
------------------------------------------------------


To load and combine the multiple netcdfs from each site into one xarray, use xarray's `open_mfdataset`.

In [9]:
ds_r = xr.open_mfdataset(f"{archive_location}/{sites[0]}/netcdf/*.nc", engine="netcdf4")
ds_r

<xarray.Dataset> Size: 11GB
Dimensions:          (time: 351, chirp_num: 100, attenuator_setting_pair: 1,
                      chirp_time: 40001)
Coordinates:
  * time             (time) datetime64[ns] 3kB 2023-01-16T00:52:00 ... 2023-1...
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (time) <U23 32kB dask.array<chunksize=(351,), meta=np.ndarray>
    folder_name      (time) <U9 13kB dask.array<chunksize=(351,), meta=np.ndarray>
    burst_number     (time) int64 3kB dask.array<chunksize=(351,), meta=np.ndarray>
    AFGain           (attenuator_setting_pair) int64 8B dask.array<chunksize=(1,), meta=np.ndarray>
    attenuator       (attenuator_setting_pair) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
    orientation      (time) <U7 10kB dask.array<chunksize=(351,), meta=np.ndarray>
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_num, attenuator_setting_pair, chirp_time) float64 11GB dask.array<chunksize=(351, 100, 1, 40001), meta=np.ndarray>
    latitude         (time) float64 3kB dask.array<chunksize=(351,), meta=np.ndarray>
    longitude        (time) float64 3kB dask.array<chunksize=(351,), meta=np.ndarray>
    battery_voltage  (time) float64 3kB dask.array<chunksize=(351,), meta=np.ndarray>
    temperature_1    (time) float64 3kB dask.array<chunksize=(351,), meta=np.ndarray>
    temperature_2    (time) float64 3kB dask.array<chunksize=(351,), meta=np.ndarray>
    chirp_stacked    (time, attenuator_setting_pair, chirp_time) float64 112MB dask.array<chunksize=(351, 1, 40001), meta=np.ndarray>
Attributes:
    processing:  Created on 2025-07-02 21:40:29.009544
    c:           300000000.0
    K:           200000000.0
    f_1:         200000000.0
    f_2:         400000000.0
    dt:          2.5e-05
    ep:          3.18
    B:           200000000.0
    f_c:         300000000.0